# Test de la Chi Cuadrado en Python

Este test es famosísimo y muy útil ya que:

* Aporta un criterio sólido para encontrar relaciones entre variables cualitativas usando las tablas de contingencia

$$
\chi^2=\sum_{i=1}^{k} \frac{(valor.observado_{i}-valor.esperado_{i})^2}{valor.esperado_{i}}
$$

El fundamento de este test es comparar la suma de diferencias al cuadrado entre lo que estamos observando, es decir, nuestros datos reales, y aquello que cabría esperar en una situación donde no existe ninguna relación entre variables.

Esto permite cuantificar la magnitud de todas estas distancias, compararlas con una distribución, que es la de 'Chi' cuadrado, y decidir si podemos afirmar que estas variables están relacionadas significativamente.

**Una de las debilidades de este test** es que no permite cuantificar cada una de las relaciones entre categorías, pero permite afirmaciones globales del tipo:

* "fumar está relacionado significativamente con el cáncer de pulmón".

Y, aun así, una exploración visual de las tablas permite determinar en qué dirección se da esta relación. después de esta breve introducción, vamos a ver un ejemplo de código sobre datos de aerolíneas

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv("./data_src/DelayedFlights.csv", nrows=350000)

In [8]:
# Marcamos semilla
np.random.seed(42)

In [9]:
# Nos quedamos solo con los vuelos que parten de HOU, SFO e IND

df = df[df["Origin"].isin(["HOU","SFO","IND"])]

In [18]:
# Sampelizamos el dataframe

df = df.sample(frac=1)

### Nuestro Objetivo Último de Análisis

El objetivo último de análisis es conocer si el aeropuerto de salida del avión tiene influencia o no en el hecho de que el avión se retrase mucho o no.

El aeropuerto de salida del avión está detallado en 'Origin'. Que el avión se retrase mucho o no, no está definido directamente, por lo que deberemos "ingenierizar" una nueva variable.

Es decir, creamos una nueva variable en nuestro dataframe de tipo True/False. Dicha variable hace referencia a si el avión se ha retrasado mucho (True) o no (False).

Para mí, "retrasarse mucho" es retrasarse más de 45 minutos

In [19]:
# Creo la variable

df["BigDelay"] = df["ArrDelay"] > 45

### Creación de una tabla de contingencia con 'crosstab'

Una vez creada dicha variable, ya tenemos los dos elementos del análisis:

* El aeropuerto de origen
* El avión ha sufrido un retraso fuerte o no.

Como puede verse se trata de dos variables categóricas y, por tanto, una tabla de contingencia (también llamada de clasificación cruzada) es una de las formas más comunes de resumir datos categóricos.


Pandas tiene su propio método para la creación de tablas de contingencia con .crosstab(···)

[Más info de Tablas de Contingencia aquí](http://halweb.uc3m.es/esp/Personal/personas/jmmarin/esp/Categor/Tema2Cate.pdf)

In [23]:
# margins=True nos permite visualizar también las frecuencias marginales, es decir, los totales por fila y por columna

pd.crosstab(index=df["BigDelay"], columns=df["Origin"], margins=True)

Origin,HOU,IND,SFO,All
BigDelay,,,,
False,2695,1246,4401,8342
True,930,639,4008,5577
All,3625,1885,8409,13919


In [26]:
tabla_contingencia = pd.crosstab(index=df["BigDelay"], columns=df["Origin"], margins=True)

### Realización del test de Chi cuadrado

In [27]:
# Importamos los paquetes:

from scipy.stats import chi2_contingency

In [32]:
# Realizamos el test en si mismo:

resultado_test_chi2 = chi2_contingency(tabla_contingencia)
resultado_test_chi2

(545.4625906718225,
 1.3425426732110207e-114,
 6,
 array([[ 2172.55190746,  1129.72699188,  5039.72110065,  8342.        ],
        [ 1452.44809254,   755.27300812,  3369.27889935,  5577.        ],
        [ 3625.        ,  1885.        ,  8409.        , 13919.        ]]))

El método chi2_contigency(···) nos devuelve una tupla con varias posiciones:

* **Pos [0]:** El estadístico, es decir, la suma de las diferencias al cuadrado.
* **Pos [1]:** El p-valor
* **Pos [2]:** ??
* **Pos [3]:** Un array 2D cuya dimensionalidad es igual a la de la tabla de contigencias. Esta, es la tabla de valores esperados y es la que vamos a comparar con nuestra tabla de valores observados (es decir tabla_contingencia). Nótese que se obtienen decimales cuando deberían ser números enteros ya que hace referencia a frecuencias absolutas pero esto son cálculos teóricos

Por tanto renombrando ligeramente y asignando nuevas variables:

In [49]:
valores_observados = tabla_contingencia
valores_esperados  = pd.DataFrame(
                                   resultado_test_chi2[3],
                                   index=valores_observados.index,
                                   columns=valores_observados.columns)

In [50]:
valores_esperados

Origin,HOU,IND,SFO,All
BigDelay,,,,
False,2172.551907,1129.726992,5039.721101,8342.0
True,1452.448093,755.273008,3369.278899,5577.0
All,3625.000000,1885.000000,8409.000000,13919.0


Ahora lo que haremos es calcular las frecuencias relativas de valores observados y esperados.

Esto lo hago porque comparar valores relativos es siempre más sencillo que comparar absolutos

In [54]:
print("*****   Valores Observados   *****")
valores_observados_rel = round(valores_observados.apply(lambda r: r/len(df)*100),2)
valores_observados_rel

*****   Valores Observados   *****


Origin,HOU,IND,SFO,All
BigDelay,,,,
False,19.36,8.95,31.62,59.93
True,6.68,4.59,28.80,40.07
All,26.04,13.54,60.41,100.00


In [53]:
print("*****   Valores Esperados   *****")
valores_esperados_rel = round(valores_esperados.apply(lambda r: r/len(df)*100),2)
valores_esperados_rel

*****   Valores Esperados   *****


Origin,HOU,IND,SFO,All
BigDelay,,,,
False,15.61,8.12,36.21,59.93
True,10.44,5.43,24.21,40.07
All,26.04,13.54,60.41,100.00


Al comparar ambas tablas vemos que los margenes se han mantenido, sino, es que algo hemos hecho mal :/

Lo que hace el test por nosotros es comparar si el 24.21 de True-SFO en valores esperados es SUFICIENTEMENTE menor que el 28.80 de True-SFO en términos estadísticos.

### Empleando el p-valor para conocer el resultado del test de hipótesis:

En este caso el "significado" que se le puede atribuir al p-valor es el de la probabilidad de que las relaciones sean mucho más extremas de lo que hemos visto en nuestros datos si comparamos con la situación donde no hay ninguna relación.

Dicho de otra forma, es un indicador de como de extraño sería ver nuestros datos sino hubiese relación entre variables.

**En cristiano:**

* Si el p-valor < 0.05 hay diferencias significativas --> Hay relación entre las variables
* Si el p-valor > 0.05 **NO HAY** diferencias significativas --> Las variables no están relacionadas.

In [55]:
# En nuestro caso concreto:

p_valor = resultado_test_chi2[1]
p_valor

1.3425426732110207e-114

Podemos afirmar que SÍ HAY relación entre variables y los retrasos de los vuelos están influenciados por los aeropuertos de origen y/o viceversa.